In [2]:
import requests
import pandas as pd
import pyarrow as pa
import pyarrow.json as pjson
import polars as pl
from tqdm.notebook import tqdm
from datetime import datetime
from time import sleep
from io import StringIO
from collections import OrderedDict
import json
from datetime import date
from pathlib import Path

content_path = Path("/Users/davidenicoli/Local_Workspace/Datasets/ARPA/PIEMONTE/")
meteo_path = content_path / "meteo"
centenaria_path = content_path / "centenaria"


def db_path(section):
    return content_path / section / "fragments"


def meta_path(section):
    return content_path / section / "meta.csv"


def dati_giornalieri(id_punto_misura, section, session: requests.Session):
    return session.get(
        f"https://utility.arpa.piemonte.it/meteoidro/dati_giornalieri_{section}/",
        params={f"fk_id_punto_misura_{section}": id_punto_misura},
    ).json()

In [2]:
def download_pages(access_url, cache_dir: Path, params={}):
    cache_dir.mkdir(parents=True, exist_ok=True)
    with requests.Session() as session:
        response = session.get(access_url, params=params).json()
        i = 0
        with open(cache_dir / f"{i}.json", "wt") as f:
            json.dump(response, f, ensure_ascii=False)
        while response["next"]:
            i += 1
            response = session.get(response["next"]).json()
            with open(cache_dir / f"{i}.json", "wt") as f:
                json.dump(response, f, ensure_ascii=False)

In [3]:
metadata_urls = {
    "sensori": "https://utility.arpa.piemonte.it/meteoidro/sensore_meteo/",
    "stazioni_meteo": "https://utility.arpa.piemonte.it/meteoidro/stazione_meteorologica/",
    "stazioni_nivo": "https://utility.arpa.piemonte.it/meteoidro/stazione_nivologica/",
    "punti_misura_meteo": "https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/",
    "stazioni_centenarie": "https://utility.arpa.piemonte.it/meteoidro/stazione_centenaria/",
    "punti_misura_centenaria": "https://utility.arpa.piemonte.it/meteoidro/punti_misura_centenaria/",
}

In [78]:
for section, url in tqdm(metadata_urls.items()):
    download_pages(url, content_path / section)

  0%|          | 0/6 [00:00<?, ?it/s]

## Sensori
Tipologia di sensore, inizio e fine misure, stazione di appartenenza

In [33]:
sensori_input = {
    "url": pl.String(),
    "id_parametro": pl.String(),
    "data_inizio": pl.String(),
    "data_fine": pl.String(),
    "quota_da_pc": pl.Float64(),
    "altezza_supporto": pl.Float64(),
    "note": pl.String(),
    "fk_id_stazione_meteorologica": pl.String(),
}
sensori = []
for file in (content_path / "sensori").glob("*.json"):
    with open(file) as f:
        sensori.extend(json.load(f)["results"])
sensori = pl.from_records(sensori, schema=sensori_input).with_columns(
    pl.col("url")
    .str.extract(r"https://utility.arpa.piemonte.it/meteoidro/sensore_meteo/([^/]+)/")
    .alias("id_sensore"),
    pl.col("fk_id_stazione_meteorologica")
    .str.extract(
        r"https://utility.arpa.piemonte.it/meteoidro/stazione_meteorologica/([^/]+)/"
    )
    .alias("id_stazione_meteo"),
)
sensori.write_csv(content_path / "sensori" / "meta.csv")

## Stazioni meteo
Metadati stazioni, con tanto di storia sensori

In [7]:
with open(content_path / "stazioni_meteo" / "0.json") as f:
    sensori_meteo = pl.from_records(
        json.load(f)["results"],
        schema_overrides={
            "codice_istat_comune": pl.String(),
            "codice_stazione": pl.String(),
        },
    )

sensori_meteo = (
    sensori_meteo.explode("sensori_meteo")
    .with_columns(
        pl.col("sensori_meteo").struct.rename_fields(
            [
                "url_sensore",
                "id_parametro",
                "data_inizio_sensore",
                "data_fine_sensore",
                "quota_da_pc",
                "altezza_supporto",
                "note_sensore",
                "fk_id_stazione_meteorologica",
            ]
        )
    )
    .unnest("sensori_meteo")
    .with_columns(
        pl.col("url_sensore")
        .str.extract(
            r"https://utility.arpa.piemonte.it/meteoidro/sensore_meteo/([^/]+)/"
        )
        .alias("id_sensore"),
        pl.col("fk_id_stazione_meteorologica")
        .str.extract(
            r"https://utility.arpa.piemonte.it/meteoidro/stazione_meteorologica/([^/]+)/"
        )
        .alias("id_stazione_meteo"),
        pl.col("fk_id_punto_misura_meteo")
        .str.extract(
            r"https://utility.arpa.piemonte.it/meteoidro/punti_misura_meteo/([^/]+)/"
        )
        .alias("id_punto_misura_meteo"),
    )
)
sensori_meteo.filter(pl.col("id_parametro").eq("TERMA")).write_csv(
    content_path / "meteo" / "meta.csv"
)

In [6]:
with open(content_path / "stazioni_centenarie" / "0.json") as f:
    stazioni_centenarie = pl.from_records(
        json.load(f)["results"],
        schema_overrides={
            "codice_istat_comune": pl.String(),
            "codice_stazione": pl.String(),
        },
    )

stazioni_centenarie = stazioni_centenarie.with_columns(
    pl.col("url")
    .str.extract(
        r"https://utility.arpa.piemonte.it/meteoidro/stazione_centenaria/([^/]+)/"
    )
    .alias("id_stazione_centenaria"),
    pl.col("fk_id_punto_misura_centenaria")
    .str.extract(
        r"https://utility.arpa.piemonte.it/meteoidro/punti_misura_centenaria/([^/]+)/"
    )
    .alias("id_punto_misura_centenaria"),
)
stazioni_centenarie.write_csv(content_path / "centenaria" / "meta.csv")

## Stazioni nivo

Anche loro hanno dei termometri

In [3]:
with open(content_path / "stazioni_nivo" / "0.json") as f:
    sensori_nivo = pl.from_records(json.load(f)["results"])

sensori_nivo = sensori_nivo.with_columns(
    pl.col("url")
    .str.extract(
        r"https://utility.arpa.piemonte.it/meteoidro/stazione_nivologica/([^/]+)/"
    )
    .alias("id_stazione_nivo"),
    pl.col("fk_id_punto_misura_nivo")
    .str.extract(
        r"https://utility.arpa.piemonte.it/meteoidro/punti_misura_nivo/([^/]+)/"
    )
    .alias("id_punto_misura_nivo"),
)
sensori_nivo.write_csv(content_path / "nivo" / "meta.csv")

In [116]:
nivo_extra = sensori_nivo.join(
    sensori_meteo,
    # left_on="id_punto_misura_nivo",
    # right_on="id_punto_misura_meteo",
    on="codice_stazione",
    how="anti",
)

## Punti misura meteo
Collezione di stazioni/sensori per una località

In [66]:
with open(
    "/Users/davidenicoli/Local_Workspace/Datasets/ARPA/PIEMONTE/punti_misura_meteo/0.json"
) as f:
    punti_misura_meteo = pl.from_records(json.load(f)["results"])

In [2]:
content = requests.get(
    "https://utility.arpa.piemonte.it/meteoidro/stazione_meteorologica/"
).content
meteo_info = json.loads(content)

content = requests.get(
    "https://utility.arpa.piemonte.it/meteoidro/stazione_centenaria/"
).content
centenaria_info = json.loads(content)

In [3]:
# Meteo metadata
meteo_interm = pl.DataFrame(
    meteo_info["results"],
    schema_overrides={
        "url": pl.Utf8(),
        "codice_istat_comune": pl.Utf8(),
        "sensori_meteo": pl.List(
            pl.Struct(
                {
                    "url": pl.Utf8(),
                    "id_parametro": pl.Utf8(),
                    "data_inizio": pl.Utf8(),
                    "data_fine": pl.Utf8(),
                    "quota_da_pc": pl.Float32(),
                    "altezza_supporto": pl.Float32(),
                    "note": pl.Utf8(),
                    "fk_id_stazione_meteorologica": pl.Utf8(),
                }
            )
        ),
    },
)
meteo_meta = (
    (
        meteo_interm.rename({"url": "url_stazione"})
        .explode("sensori_meteo")
        .with_columns(
            pl.col("sensori_meteo").struct.rename_fields(
                [
                    "url_sensore",
                    "id_parametro",
                    "data_inizio_sensore",
                    "data_fine_sensore",
                    "quota_da_pc",
                    "altezza_supporto",
                    "note_sensore",
                    "copy_url_stazione",
                ]
            )
        )
        .unnest("sensori_meteo")
        .filter(pl.col("id_parametro") == "TERMA")
    )
    .with_columns(
        pl.col("url_stazione")
        .str.split("/")
        .list.get(-2)
        .str.slice(0, 14)
        .alias("codice_punto_misura"),
    )
    .select(
        [
            "url_stazione",
            "url_sensore",
            "id_parametro",
            "data_inizio_sensore",
            "data_fine_sensore",
            "quota_da_pc",
            "altezza_supporto",
            "note_sensore",
            "codice_istat_comune",
            "codice_stazione",
            "denominazione",
            "indirizzo_localita",
            "nazione",
            "longitudine_e_wgs84_d",
            "latitudine_n_wgs84_d",
            "quota_stazione",
            "esposizione",
            "note",
            "tipo_staz",
            "data_inizio",
            "data_fine",
            "sigla_prov",
            "comune",
            "fk_id_punto_misura_meteo",
            "codice_punto_misura",
        ]
    )
)
meteo_meta.write_csv(meta_path("meteo"), quote_style="always")

# Centenaria metadata
centenarie_meta = pl.DataFrame(
    centenaria_info["results"],
).with_columns(
    pl.col("fk_id_punto_misura_centenaria")
    .str.split("/")
    .list.get(-2)
    .alias("codice_punto_misura"),
    pl.col("longitudine_e_wgs84_d").cast(pl.Float64()),
    pl.col("latitudine_n_wgs84_d").cast(pl.Float64()),
)
centenarie_meta.write_csv(meta_path("centenaria"), quote_style="always")

In [4]:
meteo_meta = pl.read_csv(meta_path("meteo"))
centenarie_meta = pl.read_csv(meta_path("centenaria"))

In [103]:
def punto_path(id_punto, section) -> Path:
    return db_path(section) / f"{id_punto}/"

def fragment_path(id_punto, first_date, last_date, section) -> Path:
    return (
        punto_path(id_punto, section) / f"{first_date.strftime(r"%Y-%m-%d")}_{last_date.strftime(r"%Y-%m-%d")}.csv"
    )

def date_extremes(results):
    dates = [datetime.strptime(r["data"], r"%Y-%m-%d") for r in results]
    return min(dates), max(dates)

def dwn_punto_misura(id_punto, session: requests.Session, section):
    pp = punto_path(id_punto, section)
    if not pp.exists():
        pp.mkdir(parents=True, exist_ok=True)
    else:
        return True
    # try:
    page = dati_giornalieri(id_punto, section, session)
    start_date, stop_date = date_extremes(page["results"])
    pl.read_json(StringIO(json.dumps(page["results"]))).drop("url").write_csv(fragment_path(id_punto, start_date, stop_date, section))
    try:
        while page["next"] is not None:
            page = session.get(page["next"]).json()
            start_date, stop_date = date_extremes(page["results"])
            pl.read_json(StringIO(json.dumps(page["results"]))).drop("url").write_csv(fragment_path(id_punto, start_date, stop_date, section))
            sleep(0.2)
        return True
    except:
        for file in pp.glob("*"):
            file.unlink()
        pp.rmdir()
        return False


In [8]:
ids = meteo_meta["codice_punto_misura"].unique().to_list()
for id in tqdm(ids):
    with requests.Session() as session:
        if not dwn_punto_misura(id, session, "meteo"):
            print("Could not download meteo data for ", id)
            sleep(5)

  0%|          | 0/311 [00:00<?, ?it/s]

In [6]:
ids = centenarie_meta["codice_punto_misura"].unique().to_list()
for id in tqdm(ids):
    with requests.Session() as session:
        if not dwn_punto_misura(id, session, "centenaria"):
            print("Could not download meteo data for ", id)
            sleep(5)

  0%|          | 0/2 [00:00<?, ?it/s]

In [108]:
ids = nivo_extra["id_punto_misura_nivo"].unique().to_list()
for id in tqdm(ids):
    with requests.Session() as session:
        if not dwn_punto_misura(id, session, "nivo"):
            print("Could not download meteo data for ", id)
            sleep(5)

  0%|          | 0/42 [00:00<?, ?it/s]

In [107]:
len(nivo_extra["id_punto_misura_nivo"].unique().to_list())

42